# 11.3 日期的范围、频率以及移动

pandas中的原生时间序列一般被认为是不规则的，也就是说，它们没有固定的频率。对于大部分应用程序而言，这是无所谓的。但是，它常常需要以某种相对固定的频率进行分析，比如每日、每月、每15分钟等（这样自然会在时间序列中引入缺失值）。幸运的是，pandas有一整套标准时间序列频率以及用于重采样、频率推断、生成固定频率日期范围的工具。例如，我们可以将之前那个时间序列转换为一个具有固定频率（每日）的时间序列，只需调用resample即可：

In [1]:
from datetime import datetime

import pandas as pd
import numpy as np

In [2]:
dates = [datetime(2011, 1, 2),  
         datetime(2011, 1, 5),
         datetime(2011, 1, 7),  
         datetime(2011, 1, 8),
         datetime(2011, 1, 10), 
         datetime(2011, 1, 12)
        ]


ts = pd.Series(np.random.randn(6), index=dates)

In [4]:
ts

2011-01-02    0.165659
2011-01-05   -1.552404
2011-01-07   -0.490445
2011-01-08    1.891984
2011-01-10    0.641196
2011-01-12   -0.538430
dtype: float64

In [6]:
resampler = ts.resample('D')

resampler

字符串“D”是每天的意思。

频率的转换（或重采样）是一个比较大的主题，稍后将专门用一节来进行讨论（11.6小节）。这里，我将告诉你如何使用基本的频率和它的倍数。

## 生成日期范围

虽然我之前用的时候没有明说，但你可能已经猜到pandas.date_range可用于根据指定的频率生成指定长度的DatetimeIndex：

In [7]:
index = pd.date_range('2012-04-01', '2012-06-01')

index

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20',
               '2012-04-21', '2012-04-22', '2012-04-23', '2012-04-24',
               '2012-04-25', '2012-04-26', '2012-04-27', '2012-04-28',
               '2012-04-29', '2012-04-30', '2012-05-01', '2012-05-02',
               '2012-05-03', '2012-05-04', '2012-05-05', '2012-05-06',
               '2012-05-07', '2012-05-08', '2012-05-09', '2012-05-10',
               '2012-05-11', '2012-05-12', '2012-05-13', '2012-05-14',
               '2012-05-15', '2012-05-16', '2012-05-17', '2012-05-18',
               '2012-05-19', '2012-05-20', '2012-05-21', '2012-05-22',
               '2012-05-23', '2012-05-24', '2012-05-25', '2012-05-26',
      

默认情况下，date_range会产生按天计算的时间点。如果只传入起始或结束日期，那就还得传入一个表示一段时间的数字：

In [8]:
pd.date_range(start='2012-04-01', periods=20)

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20'],
              dtype='datetime64[ns]', freq='D')

In [9]:
pd.date_range(end='2012-06-01', periods=20)

DatetimeIndex(['2012-05-13', '2012-05-14', '2012-05-15', '2012-05-16',
               '2012-05-17', '2012-05-18', '2012-05-19', '2012-05-20',
               '2012-05-21', '2012-05-22', '2012-05-23', '2012-05-24',
               '2012-05-25', '2012-05-26', '2012-05-27', '2012-05-28',
               '2012-05-29', '2012-05-30', '2012-05-31', '2012-06-01'],
              dtype='datetime64[ns]', freq='D')

起始和结束日期定义了日期索引的严格边界。例如，如果你想要生成一个由每月最后一个工作日组成的日期索引，可以传入"BM"频率（表示business end of month，表11-4是频率列表），这样就只会包含时间间隔内（或刚好在边界上的）符合频率要求的日期：

In [10]:
pd.date_range('2000-01-01', '2000-12-01', freq='BM')

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-28',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-29', '2000-10-31', '2000-11-30'],
              dtype='datetime64[ns]', freq='BM')

表11-4 基本的时间序列频率（不完整）

date_range默认会保留起始和结束时间戳的时间信息（如果有的话）：

In [11]:
pd.date_range('2012-05-02 12:56:31', periods=5)

DatetimeIndex(['2012-05-02 12:56:31', '2012-05-03 12:56:31',
               '2012-05-04 12:56:31', '2012-05-05 12:56:31',
               '2012-05-06 12:56:31'],
              dtype='datetime64[ns]', freq='D')

有时，虽然起始和结束日期带有时间信息，但你希望产生一组被规范化（normalize）到午夜的时间戳。normalize选项即可实现该功能：

In [12]:
pd.date_range('2012-05-02 12:56:31', periods=5, normalize=True)

DatetimeIndex(['2012-05-02', '2012-05-03', '2012-05-04', '2012-05-05',
               '2012-05-06'],
              dtype='datetime64[ns]', freq='D')

## 频率和日期偏移量

pandas中的频率是由一个基础频率（base frequency）和一个乘数组成的。基础频率通常以一个字符串别名表示，比如"M"表示每月，"H"表示每小时。对于每个基础频率，都有一个被称为日期偏移量（date offset）的对象与之对应。例如，按小时计算的频率可以用Hour类表示：

In [13]:
from pandas.tseries.offsets import Hour, Minute

In [14]:
hour = Hour()

hour

<Hour>

传入一个整数即可定义偏移量的倍数：

In [16]:
four_hours = Hour(4)

four_hours

<4 * Hours>

一般来说，无需明确创建这样的对象，只需使用诸如"H"或"4H"这样的字符串别名即可。在基础频率前面放上一个整数即可创建倍数：

In [17]:
pd.date_range('2000-01-01', '2000-01-03 23:59', freq='4h')

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 04:00:00',
               '2000-01-01 08:00:00', '2000-01-01 12:00:00',
               '2000-01-01 16:00:00', '2000-01-01 20:00:00',
               '2000-01-02 00:00:00', '2000-01-02 04:00:00',
               '2000-01-02 08:00:00', '2000-01-02 12:00:00',
               '2000-01-02 16:00:00', '2000-01-02 20:00:00',
               '2000-01-03 00:00:00', '2000-01-03 04:00:00',
               '2000-01-03 08:00:00', '2000-01-03 12:00:00',
               '2000-01-03 16:00:00', '2000-01-03 20:00:00'],
              dtype='datetime64[ns]', freq='4H')

大部分偏移量对象都可通过加法进行连接：

In [18]:
Hour(2) + Minute(30)

<150 * Minutes>

同理，你也可以传入频率字符串（如"2h30min"），这种字符串可以被高效地解析为等效的表达式：

In [19]:
pd.date_range('2000-01-01', periods=10, freq='1h30min')

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 01:30:00',
               '2000-01-01 03:00:00', '2000-01-01 04:30:00',
               '2000-01-01 06:00:00', '2000-01-01 07:30:00',
               '2000-01-01 09:00:00', '2000-01-01 10:30:00',
               '2000-01-01 12:00:00', '2000-01-01 13:30:00'],
              dtype='datetime64[ns]', freq='90T')

有些频率所描述的时间点并不是均匀分隔的。例如，"M"（日历月末）和"BM"（每月最后一个工作日）就取决于每月的天数，对于后者，还要考虑月末是不是周末。由于没有更好的术语，我将这些称为锚点偏移量（anchored offset）。

表11-4列出了pandas中的频率代码和日期偏移量类。

>笔记：用户可以根据实际需求自定义一些频率类以便提供pandas所没有的日期逻辑，但具体的细节超出了本书的范围。             

表11-4 时间序列的基础频率

## WOM日期

WOM（Week Of Month）是一种非常实用的频率类，它以WOM开头。它使你能获得诸如“每月第3个星期五”之类的日期：

In [21]:
rng = pd.date_range('2012-01-01', '2012-09-01', freq='WOM-3FRI')

list(rng)

[Timestamp('2012-01-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-02-17 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-03-16 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-04-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-05-18 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-06-15 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-07-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-08-17 00:00:00', freq='WOM-3FRI')]

## 移动（超前和滞后）数据

移动（shifting）指的是沿着时间轴将数据前移或后移。Series和DataFrame都有一个shift方法用于执行单纯的前移或后移操作，保持索引不变：

In [24]:
ts = pd.Series(np.random.randn(4),
               index=pd.date_range('1/1/2000', periods=4, freq='M'))

ts

2000-01-31   -0.408789
2000-02-29    0.345932
2000-03-31   -0.015869
2000-04-30    0.387771
Freq: M, dtype: float64

In [25]:
ts.shift(2)

2000-01-31         NaN
2000-02-29         NaN
2000-03-31   -0.408789
2000-04-30    0.345932
Freq: M, dtype: float64

In [28]:
ts.shift(-2)

2000-01-31   -0.015869
2000-02-29    0.387771
2000-03-31         NaN
2000-04-30         NaN
Freq: M, dtype: float64

当我们这样进行移动时，就会在时间序列的前面或后面产生缺失数据。

shift通常用于计算一个时间序列或多个时间序列（如DataFrame的列）中的百分比变化。可以这样表达：

In [29]:
ts / ts.shift(1) - 1

2000-01-31          NaN
2000-02-29    -1.846236
2000-03-31    -1.045874
2000-04-30   -25.435101
Freq: M, dtype: float64

由于单纯的移位操作不会修改索引，所以部分数据会被丢弃。因此，如果频率已知，则可以将其传给shift以便实现对时ts.shift(2, freq='M')间戳进行位移而不是对数据进行简单位移：

In [30]:
ts.shift(2, freq='M')

2000-03-31   -0.408789
2000-04-30    0.345932
2000-05-31   -0.015869
2000-06-30    0.387771
Freq: M, dtype: float64

这里还可以使用其他频率，于是你就能非常灵活地对数据进行超前和滞后处理了：

In [31]:
ts.shift(3, freq='D')

2000-02-03   -0.408789
2000-03-03    0.345932
2000-04-03   -0.015869
2000-05-03    0.387771
dtype: float64

In [32]:
ts.shift(1, freq='90T')

2000-01-31 01:30:00   -0.408789
2000-02-29 01:30:00    0.345932
2000-03-31 01:30:00   -0.015869
2000-04-30 01:30:00    0.387771
dtype: float64

## 通过偏移量对日期进行位移

pandas的日期偏移量还可以用在datetime或Timestamp对象上：

In [36]:
from pandas.tseries.offsets import Day, MonthEnd


now = datetime(2023, 3, 1)

now

datetime.datetime(2023, 3, 1, 0, 0)

In [37]:
now + 3 * Day()

Timestamp('2023-03-04 00:00:00')

如果加的是锚点偏移量（比如MonthEnd），第一次增量会将原日期向前滚动到符合频率规则的下一个日期：

In [38]:
now + MonthEnd()

Timestamp('2023-03-31 00:00:00')

In [39]:
now + MonthEnd(2)

Timestamp('2023-04-30 00:00:00')

通过锚点偏移量的rollforward和rollback方法，可明确地将日期向前或向后“滚动”：

In [42]:
offset = MonthEnd()

offset.rollforward(now)

Timestamp('2023-03-31 00:00:00')

In [43]:
offset.rollback(now)

Timestamp('2023-02-28 00:00:00')

日期偏移量还有一个巧妙的用法，即结合groupby使用这两个“滚动”方法：

In [45]:
ts = pd.Series(np.random.randn(20),
               index=pd.date_range('1/15/2023', periods=20, freq='4d')
              )

ts

2023-01-15    0.838336
2023-01-19    0.100061
2023-01-23   -0.330472
2023-01-27   -1.379282
2023-01-31   -0.473736
2023-02-04    1.267700
2023-02-08    0.015096
2023-02-12    0.840505
2023-02-16    0.915148
2023-02-20   -1.363342
2023-02-24   -1.029827
2023-02-28   -0.366487
2023-03-04    0.618433
2023-03-08   -0.023712
2023-03-12   -1.085482
2023-03-16    0.534977
2023-03-20    0.393094
2023-03-24   -0.162449
2023-03-28    0.837970
2023-04-01    0.351977
Freq: 4D, dtype: float64

In [46]:
ts.groupby(offset.rollforward).mean()

2023-01-31   -0.249019
2023-02-28    0.039828
2023-03-31    0.158976
2023-04-30    0.351977
dtype: float64

当然，更简单、更快速地实现该功能的办法是使用resample（11.6小节将对此进行详细介绍）：

In [47]:
ts.resample('M').mean()

2023-01-31   -0.249019
2023-02-28    0.039828
2023-03-31    0.158976
2023-04-30    0.351977
Freq: M, dtype: float64